In [37]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate, cross_val_score, train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, r2_score


from bikesharing.ml_logic.model import get_folds, train_test_indices
from bikesharing.interface.main import *



In [38]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression


In [11]:
# # Some test data
# X = pd.DataFrame({'x': [300, 170, 288, 360, 319, 330, 520, 345, 399, 479]})
# y = pd.DataFrame({'y': [305000, 270000, 360000, 370000, 379000, 405000, 407500, 450000, 450000, 485000]})
# X_seq = np.linspace(min(X['x']), max(X['x']), 300).reshape(-1, 1)

In [7]:
# preprocess()


Load preprocessed data from local CSV...


(       temperature_2m  relativehumidity_2m  apparent_temperature  \
 0            0.355408             1.000000              0.342007   
 1            0.357616             0.987013              0.340149   
 2            0.359823             1.000000              0.336431   
 3            0.359823             0.987013              0.332714   
 4            0.359823             1.000000              0.332714   
 ...               ...                  ...                   ...   
 35035        0.426049             0.779221              0.405204   
 35036        0.412804             0.779221              0.395911   
 35037        0.410596             0.753247              0.390335   
 35038        0.417219             0.714286              0.390335   
 35039        0.415011             0.701299              0.394052   
 
        windspeed_10m  precipitation  hour_sin  hour_cos  month_sin  month_cos  \
 0           0.227848       0.017391  0.629410  0.982963       0.75   0.933013   
 1    

In [44]:
df_X = pd.read_csv('~/.lewagon/bikesharing/data/processed/X_processed_from_2019_to_2022.csv')
df_y = pd.read_csv('~/.lewagon/bikesharing/data/processed/y_processed_from_2019_to_2022.csv')


In [45]:
df_X.columns

Index(['temperature_2m', 'relativehumidity_2m', 'apparent_temperature',
       'windspeed_10m', 'precipitation', 'hour_sin', 'hour_cos', 'month_sin',
       'month_cos', 'day_sin', 'day_cos', 'is_holiday', 'is_weekend'],
      dtype='object')

In [72]:
df_X_y = pd.concat((df_X, df_y) , axis=1)

In [69]:
X = df_X[['temperature_2m', 'relativehumidity_2m', 'apparent_temperature',
       'windspeed_10m', 'precipitation']]
y = df_y['Maxvorstadt']

In [59]:
districts = ['Altstadt-Lehel', 'Au - Haidhausen',
            'Aubing-Lochhausen-Langwied', 
            'Berg am Laim', 'Bogenhausen',
            'Feldmoching', 
            'Hadern', 'Harlaching', 
            'Hasenbergl-Lerchenau Ost',
            'Laim', 'Lochhausen', 'Ludwigsvorstadt-Isarvorstadt', 'Maxvorstadt',
            'Milbertshofen-Am Hart', 'Moosach', 'Neuhausen-Nymphenburg',
            'Obergiesing', 
            'Obermenzing', 
            'Obersendling', 'Pasing',
            'Pasing-Obermenzing', 'Ramersdorf-Perlach', 'Schwabing-Freimann',
            'Schwabing-West', 'Schwanthalerhöhe', 'Sendling', 'Sendling-Westpark',
            'Südgiesing', 'Thalkirchen', 'Trudering', 'Trudering-Riem',
            'Untergiesing', 'Untergiesing-Harlaching', 
            'Untermenzing-Allach'
        ]

In [ ]:
# LOOP THROUGH ALL DISTRICTS

In [62]:
# X = df_X[['temperature_2m', 'relativehumidity_2m', 'apparent_temperature',
#        'windspeed_10m', 'precipitation']]
# y = df_y['Maxvorstadt']

In [63]:
df_X_y = pd.concat((df_X, df_y) , axis=1)

In [64]:
fold_dfs = get_folds(df_X_y, fold_length=FOLD_LENGTH, fold_stride=FOLD_STRIDE)

In [73]:
from sklearn.metrics import r2_score,mean_squared_error

In [79]:
models = []
district_score_means = []

for dist in districts:
    scores_list = []

    indices = [0, 1, 2, 3]
        
    for id in range(4):
        test_df = fold_dfs[id]
        train_folds = [fold_dfs[i] for i in indices if i != id]
        
        train_df = pd.concat(train_folds, axis=0)
        X_test = test_df.drop(columns= districts)
        y_test = test_df[dist]
        X_train = train_df.drop(columns= districts)
        y_train = train_df[dist]
        
        
        degree = 2 
        poly_features = PolynomialFeatures(degree=degree)
        X_poly = poly_features.fit_transform(X_train)
        
        model = LinearRegression()
        model.fit(X_poly, y)
    
        # y_pred = [x if x >= 0 else 0 for x in model.predict(X_test)]
        y_pred = model.predict(X_test)
        scores = r2_score(y_test , y_pred)
        # scores = np.sqrt(mean_squared_error(y_test, model.predict(X_test)))
        scores_list.append(scores)
        print(f"{id} >>> scores={scores_list}")
        
        
    scores_array = np.array(scores)
    district_score_means.append(np.mean(scores_array.flatten()))

ValueError: Found input variables with inconsistent numbers of samples: [26280, 35040]

In [ ]:
# Predict using the trained model
y_pred = model.predict(X_poly)

# Calculate the mean squared error
mse = mean_squared_error(y, y_pred)
# r2_score = r2_score(y, y_pred)

print("Mean Squared Error:", mse)
# print("R2:", r2_score)

In [55]:
############

In [67]:
X = df_X[['temperature_2m', 'relativehumidity_2m', 'apparent_temperature',
       'windspeed_10m', 'precipitation']]
y = df_y['Maxvorstadt']

degree = 2 
poly_features = PolynomialFeatures(degree=degree)
X_poly = poly_features.fit_transform(X)

# Train a linear regression model
model = LinearRegression()
model.fit(X_poly, y)

# Predict using the trained model
y_pred = model.predict(X_poly)

# Calculate the mean squared error
mse = mean_squared_error(y, y_pred)
rmse = np.sqrt(mean_squared_error(y, y_pred))
# r2_score = r2_score(y, y_pred)

# print("Mean Squared Error:", mse)
print("Squared Mean Squared Error:", rmse)
# print("R2:", r2_score)

Squared Mean Squared Error: 8.695652310991832
